In [ ]:
import os
import time

from TwitterEventDetector import *

In [ ]:
print('Started at:',time.ctime())

## Parameters

In [ ]:
original_tweet_dir = '../data/original_tweets/' # end with '/'
clean_tweet_dir = '../data/cleaned_tweets/without_retweets/' # end with '/'
subwindow_dir = '../data/cleaned_tweets/without_retweets/2012-10-12/' # each file is a subwindow in this folder
event_output_dir = '../Results/'
wiki_titles_file='../data/enwiki-titles-unstemmed.txt'
seg_prob_file='../data/seg_prob_2012_Oct_11-22.json'
wiki_Qs_file='../data/WikiQsEng_non_zero_processed.json'

# Tweet Cleaning (preprocessing)
remove_retweets=True

# segmentation
max_segment_length=4
hashtag_wt=3
entities_only = False # if True, then use only hashtags and name_mentions

# bursty segment extraction
default_seg_prob=0.0000001 # if seg prob not found in file
use_retweet_count=True
use_followers_count=True

# clustering
n_neighbors=4
threshold=4 # for news_worthiness

## Initialization

In [ ]:
ted = TwitterEventDetector(wiki_titles_file, seg_prob_file, wiki_Qs_file, remove_retweets, max_segment_length,
                           hashtag_wt, use_retweet_count, use_followers_count, default_seg_prob, entities_only)

## Tweet Cleaning

In [ ]:
#ted.clean_tweets_in_directory(original_tweet_dir, clean_tweet_dir)

## Segment tweets and create TimeWindow

In [ ]:
print('Reading SubWindows')

subwindow_files = [f.name for f in os.scandir(subwindow_dir) if f.is_file()]

subwindows = []
for subwindow_name in subwindow_files[1:4]: # read given subwindows
    print('SubWindow:',subwindow_name)
    sw = ted.read_subwindow(subwindow_dir + subwindow_name)
    subwindows.append(sw)

print('Done')    

In [ ]:
tw = TimeWindow(subwindows)
print(tw)

In [ ]:
#next_subwindow = ted.read_subwindow(subwindow_dir + subwindow_files[4])
#tw.advance_window(next_subwindow)
#print(tw)

## Bursty Segment Extraction

In [ ]:
bursty_segment_weights, segment_newsworthiness = ted.bse.get_bursty_segments(tw)

## Clustering Bursty Segments

In [ ]:
seg_sim = get_seg_similarity(bursty_segment_weights, tw)

In [ ]:
events = get_events(bursty_segment_weights, segment_newsworthiness, seg_sim, n_neighbors)

## Save Events to file

In [ ]:
# output_dir = event_output_dir + time.ctime().replace(':','-') + '/' # unique output directory each time
# os.mkdir(output_dir)

# event_no = 0
# for e,event_worthiness in events:
#     event_no += 1
#     print('\nEVENT:', event_no, 'News Worthiness:', event_worthiness) 
#     f = open(output_dir + str(event_no) + '.txt', 'w')
#     f.write(str(e)+' '+str(event_worthiness)+'\n\n')
#     for seg_name in e:
#         print(seg_name) 
#         f.write('SEGMENT:' + seg_name+'\n')
#         for text in set(tw.get_tweets_containing_segment(seg_name)):
#             f.write(text+'\n')
#         f.write('-----------------------------------------------------------\n')
#     f.close()

In [ ]:
print('Ended at:',time.ctime())

In [ ]:
for n_neighbors in [3,4,5]:
    print('No. of Neighbors', n_neighbors)
    events = get_events(bursty_segment_weights, segment_newsworthiness, seg_sim, n_neighbors)
    event_no = 0
    for e,event_worthiness in events:
        event_no += 1
        print(event_no, event_worthiness, e)
    print('-----------------------------------------------------------------------------------------------')